# Exploratory Data Analysis (EDA) - Lufemage Data Lab

**Objective:** Perform an initial exploration of the sales dataset to understand its structure, identify preliminary patterns, and detect potential data quality issues.

**Steps:**
1. Data loading.
2. General review (structure, data types).
3. Basic cleaning and preprocessing.
4. Descriptive statistical analysis.
5. Initial grouping and segmentation.


In [25]:
# Import esentials libraries
import pandas as pd
import numpy as np

# Define the path to the data file
DATA_PATH = "../data/raw_sales_data.csv"

# Upload the dataset
df = pd.read_csv(DATA_PATH)

# Check if the dataframe is not empty
if not bool(df.empty):
    print("Dataframe has been uploaded successfully.")
else:
    print("Failed to upload the dataframe.")


Dataframe has been uploaded successfully.


## 1. General Dataset Review

We will use the `.info()`, `.shape`, and `.describe()` commands to obtain a comprehensive overview of our data.  
These functions provide key insights into the dataset’s structure, including column types, the number of records


In [26]:
# View the shape of the DataFrame (rows, columns)
print(f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.")

# Get information about data types and null values
print("\nDataFrame information:")
df.info()

The dataset has 5000 rows and 6 columns.

DataFrame information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_cliente          5000 non-null   object 
 1   fecha_compra        5000 non-null   object 
 2   monto               5000 non-null   float64
 3   categoria_producto  5000 non-null   object 
 4   ciudad              5000 non-null   object 
 5   metodo_pago         5000 non-null   object 
dtypes: float64(1), object(5)
memory usage: 234.5+ KB


## 2. Data Cleaning and Preprocessing

Based on the initial review, the first step in this stage is to correct the data type of the `fecha_compra` column.  
Additionally, the dataset will be checked for duplicate or missing values to ensure data consistency and integrity.


In [27]:
# Convert 'fecha_compra' column to datetime format
df["fecha_compra"] = pd.to_datetime(df["fecha_compra"])

# Check for null values in each column
print(f"\nNull values in each column:\n{df.isnull().sum()}")

# Check for duplicate rows
print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

# Back to initial info after checks
print("\nDataframe info after initial checks: ")
df.info()


Null values in each column:
id_cliente            0
fecha_compra          0
monto                 0
categoria_producto    0
ciudad                0
metodo_pago           0
dtype: int64

Number of duplicate rows: 0

Dataframe info after initial checks: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_cliente          5000 non-null   object        
 1   fecha_compra        5000 non-null   datetime64[ns]
 2   monto               5000 non-null   float64       
 3   categoria_producto  5000 non-null   object        
 4   ciudad              5000 non-null   object        
 5   metodo_pago         5000 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 234.5+ KB


## 3. Statistical Analysis with NumPy and Pandas

With the dataset cleaned and properly formatted, basic statistical measures will be calculated for the `monto` column.  
This step provides an overview of the distribution and variability of sales amounts within the dataset.


In [28]:
# Display descriptive statistics for the 'monto' column
print("Descriptive statistics with pandas: ")
print(df["monto"].describe())

# Display descriptive statistics using numpy
# Extract 'monto' column as a numpy array
montos_np = df["monto"].to_numpy()

print("\n--- Statistics with NumPy ---")
print(f"Average sale: ${np.mean(montos_np):.2f}")
print(f"Median sale: ${np.median(montos_np):.2f}")
print(f"Standard deviation: ${np.std(montos_np):.2f}")
print(f"Maximum sale: ${np.max(montos_np):.2f}")
print(f"Minimum sale: ${np.min(montos_np):.2f}")


Descriptive statistics with pandas: 
count    5000.000000
mean      298.729818
std       209.944712
min         2.250000
25%       145.165000
50%       251.610000
75%       402.035000
max      1581.700000
Name: monto, dtype: float64

--- Statistics with NumPy ---
Average sale: $298.73
Median sale: $251.61
Standard deviation: $209.92
Maximum sale: $1581.70
Minimum sale: $2.25


## 4. Grouping and Segmentation

In this section, simple business questions will be addressed by grouping the data:
- How much was sold by product category?
- How are sales distributed by city?
- Which payment method is the most popular?


In [29]:
# Analyze total sales by product category
sales_by_category = df.groupby("categoria_producto")["monto"].sum().sort_values(ascending=False)
print("\nTotal sales by product category:\n", sales_by_category)

print("\n" + "-"*30 + "\n")

# Numbers of transactions and average amount by city
transactions_by_city = df.groupby("ciudad")["monto"].agg(["count", "mean", "sum"]).sort_values(by="sum", ascending=False)
print("Transactions and average amount by city:\n", transactions_by_city)

print("\n" + "-"*30 + "\n")

# Use of payment methods
payment_method_usage = df["metodo_pago"].value_counts()
print("Payment method usage:\n", payment_method_usage)


Total sales by product category:
 categoria_producto
Electrónica    359907.01
Ropa           303603.92
Hogar          300757.72
Alimentos      210285.62
Libros         165509.24
Juguetes       153585.58
Name: monto, dtype: float64

------------------------------

Transactions and average amount by city:
               count        mean        sum
ciudad                                    
Bogotá         2021  306.571024  619580.04
Medellín       1218  298.256798  363276.78
Cali            765  294.070078  224963.61
Cartagena       520  286.887019  149181.25
Barranquilla    476  287.074391  136647.41

------------------------------

Payment method usage:
 metodo_pago
Tarjeta de Crédito    2521
PSE                   1226
Billetera digital      783
Efectivo               470
Name: count, dtype: int64


## 5. Outlier Detection in 'monto'

An outlier is a sale that is unusually high or low compared to the rest of the data.  
The Interquartile Range (IQR) method will be used to identify these anomalous values.


In [30]:
Q1 = df["monto"].quantile(0.25)
Q3 = df["monto"].quantile(0.75)
IQR = Q3 - Q1

superior_limit = Q3 + 1.5 * IQR
inferior_limit = Q1 - 1.5 * IQR

# Filter the DataFrame to show only outliers
outliers = df[(df["monto"] > superior_limit) | (df["monto"] < inferior_limit)]

print(f"{len(outliers)} outlier sales were detected (very high amounts).")
print("Examples of outliers:")
outliers.head()

158 outlier sales were detected (very high amounts).
Examples of outliers:


,id_cliente,fecha_compra,monto,categoria_producto,ciudad,metodo_pago
82,66e4cfa4-597c-400b-92fb-6a91ba068c87,2023-12-24 16:14:49,976.68,Juguetes,Bogotá,Tarjeta de Crédito
198,16ea0e70-f038-4795-80a3-b8bda21e9004,2024-11-27 03:23:55,1062.73,Juguetes,Bogotá,Tarjeta de Crédito
208,3e76fabb-a263-4743-a7ae-4a7cf97bc1f1,2024-03-25 23:02:58,919.05,Ropa,Medellín,Tarjeta de Crédito
284,cb7f83fd-0e00-4ba4-9d8a-9f2ec1e7d8fe,2024-08-10 09:45:15,1451.33,Juguetes,Bogotá,Tarjeta de Crédito
310,f721f396-4b2c-4662-9978-6411b222c2df,2024-06-30 20:00:59,820.46,Ropa,Cartagena,Tarjeta de Crédito


## 6. Identification of Customers and Days with the Highest Purchase Volume


In [ ]:
# Top 5 customers by total purchase amount
top_clients = df.groupby("id_cliente")["monto"].sum().nlargest(5)
print("Top 5 customers by total purchase amount:\n", top_clients)

print("\n" + "-"*30 + "\n")

# Top 5 days by total sales amount
# Create a temporary column with only the date (without time)
df["fecha"] = df["fecha_compra"].dt.date
top_days = df.groupby("fecha")["monto"].sum().nlargest(5)
print("Top 5 days by total sales:\n", top_days)

# Remove the temporary column no longer needed
df = df.drop(columns=["fecha"])


Top 5 customers by total purchase amount:
 id_cliente
181a76f3-82dc-4ea2-800b-75901817259b    4746.36
5ee69e4f-8a1c-4aff-bc5a-d399b6e36665    4132.85
8422dfda-6fce-4db1-82f1-d946ba8b7adb    4002.10
f8831983-5198-46f0-bee7-9bccca0e9b20    3971.86
8b3e86c4-12b8-48ee-8d2f-fb3fa65dd89f    3934.73
Name: monto, dtype: float64

------------------------------

Top 5 days by total sales:
 date
2024-04-16    6186.41
2025-01-12    5811.80
2025-03-04    5026.86
2025-08-30    4960.96
2025-05-06    4919.58
Name: monto, dtype: float64
